In [1]:
import pandas as pd
import re
import requests
from tqdm import tqdm_notebook as tqdm

## Candidate images

In [6]:
images = pd.read_csv('../data/thirdParty_images.csv', dtype=object)

In [30]:
popularity = pd.read_csv('../data/community_images.csv', dtype=object)

In [17]:
community = pd.read_csv('../data/community_tags.zip', dtype=object)

In [73]:
community['image'] = community.apply(lambda d: str(d['slug']).replace('/',':')+':'+str(d['tag']), axis=1)

In [76]:
community = community[['image','last_updated']]

In [10]:
images.groupby('base').count()

image
base         
node    61007
python  21774
ruby     3467

In [14]:
images

image  base
0                              allenday:kiwi:latest  node
1                           amouat:dnmonster:latest  node
2               base2s:blueprint-mock-server:latest  node
3                               base2s:dredd:latest  node
4                     cdmackie:insight-dash2:latest  node
...                                             ...   ...
86243                     pivotalrabbitmq:ci:latest  ruby
86244          pivotalrabbitmq:dotnet-legacy:latest  ruby
86245                         vjohnk:redmine:latest  ruby
86246                          wuyue:redmine:latest  ruby
86247  yukimochi:mastodon-assets-precompiler:latest  ruby

[86248 rows x 2 columns]

### check the popularity of candidate images

In [18]:
images['slug'] = images['image'].apply(lambda x: x.split(':')[0]+'/'+x.split(':')[1])
len(images)

86248

In [19]:
images = (images
         .merge(popularity,
               on = 'slug',
               how = 'left'))
images['popularity'] = images['popularity'].apply(int)
len(images)

86248

In [7]:
# images = (images
#          .merge(community,
#                on = 'image',
#                how = 'left'))
# images = images.query('last_updated > "2019"')
# len(images)

# FOR NOW THIS IS SKIPPED

In [8]:
images.groupby('base').count()

image   slug  popularity
base                            
node    61007  61007       61007
python  21774  21774       21774
ruby     3467   3467        3467

In [29]:
images.popularity.sum()

971906843

In [22]:
images2 = (images
          .sort_values('popularity', ascending=False)
          .groupby('base')
          .head(1100)
          .drop('image', axis=1)
          .rename(columns={'slug':'image'})
         )
images2.shape

(3300, 3)

In [23]:
images3 = (images
          .sort_values('popularity', ascending=False)
          .groupby('base')
          .head(1500)
          .drop('image', axis=1)
          .rename(columns={'slug':'image'})
         )
images3.shape

(4500, 3)

In [123]:
images = images3[~images3.image.isin(images2.image)]

In [125]:
len(images)

1200

In [10]:
images3.to_csv('../data/to_pull.csv', index=False)

## Use the DockerHub Api to know when they were last updated

In [12]:
def download_url(url):
    ''' Download an url and returns the content'''
    
    response = requests.get(url)
    webContent  = response.json()
    return webContent

In [13]:
def image_metadata(url):
    ''' Parse a json file from url  '''
    data  = download_url(url)
    return data['last_updated']

In [ ]:
if False:
    images['last_updated'] = ''
    for image in tqdm(images.image, total= len(images)) :
        url = "https://registry.hub.docker.com/v2/repositories/"+image+"/tags/latest"
        try:
            date = image_metadata(url)
            images.loc[images.image==image, 'last_updated'] = date
        except:
            images.loc[images.image==image, 'last_updated'] = pd.np.nan

In [ ]:
images.to_csv('../data/to_pull.csv', index=False)